In [1]:
from confluent_kafka import Consumer
from confluent_kafka.admin import AdminClient, NewTopic
import time
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer


In [13]:
# List all past topics
KAFKA_BOOTSTRAP_SERVERS = "localhost:29092"
consumer_conf = {
        "bootstrap.servers": KAFKA_BOOTSTRAP_SERVERS,
        "group.id": "anygroup",
        "auto.offset.reset": "earliest",
        "security.protocol": "plaintext",
        "enable.auto.commit": False,
    }

consumer = Consumer(consumer_conf)
list(consumer.list_topics().topics)

['keda-scalling', '__consumer_offsets']

In [12]:
# Make new topics
topic = "keda-scalling"
admin_conf = {
        "bootstrap.servers": KAFKA_BOOTSTRAP_SERVERS,
    }
admin_client = AdminClient(admin_conf)
topic_list = []
topic_list.append(NewTopic(topic=topic, num_partitions=5, replication_factor=1))
resp = admin_client.create_topics(new_topics=topic_list, validate_only=False)
time.sleep(5)
if resp.get(topic).result() is None:
    print("topic created!")
else:
    print(resp.get(topic).result())

topic created!


In [16]:
# producer script 
import json
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))



producer_conf = {
    "bootstrap.servers": KAFKA_BOOTSTRAP_SERVERS,
    "value.serializer": StringSerializer(),
}
producer = SerializingProducer(producer_conf)

for i in range(10):
    producer.poll(0)
    data = {
        "name" : f"name_{i}",
        "address" :  "LOL"
    }
    res = producer.produce(topic, value=json.dumps(data), on_delivery=delivery_report)
producer.flush()

Message delivered to keda-scalling [0]
Message delivered to keda-scalling [0]
Message delivered to keda-scalling [0]
Message delivered to keda-scalling [1]
Message delivered to keda-scalling [1]
Message delivered to keda-scalling [2]
Message delivered to keda-scalling [2]
Message delivered to keda-scalling [2]
Message delivered to keda-scalling [3]
Message delivered to keda-scalling [3]


0